pd.set_option("display.max_rows", None) # 행이 많아도 자르지 않음

In [12]:
import pandas as pd
pd.set_option("display.max_rows", None)

# -------------------------------------------------------------------
df = pd.read_csv("../blinkit-dataset/protocol/blinkit_customers.csv")

df["area"].value_counts().reset_index()

,area,count
0,Jalna,18
1,Deoghar,17
2,Bathinda,17
3,Orai,17
4,Ratlam,16
5,Ghaziabad,16
6,Kishanganj,15
7,Kadapa,14
8,Nandyal,14
9,Kalyan-Dombivli,14


In [13]:
df["area"].unique()

array(['Udupi', 'Aligarh', 'Begusarai', 'Kozhikode', 'Ichalkaranji',
       'Visakhapatnam', 'Gwalior', 'Orai', 'Buxar', 'Malegaon', 'Katni',
       'Fatehpur', 'Hyderabad', 'Bhiwandi', 'Mira-Bhayandar', 'Amravati',
       'Kochi', 'Tiruchirappalli', 'Hubli–Dharwad', 'Nagercoil', 'Alwar',
       'Bardhaman', 'Nandyal', 'Hapur', 'Nanded', 'Ramgarh',
       'Bihar Sharif', 'Jorhat', 'Raipur', 'Kollam', 'Ratlam', 'Nellore',
       'Warangal', 'Kottayam', 'Tenali', 'Jaipur', 'Patiala',
       'Maheshtala', 'Ludhiana', 'Ongole', 'Katihar', 'Hospet', 'Siwan',
       'Dehradun', 'North Dumdum', 'Jamalpur', 'Parbhani', 'Srinagar',
       'Kolkata', 'Haridwar', 'Kavali', 'Junagadh', 'Karaikudi',
       'Kishanganj', 'Jaunpur', 'Madhyamgram', 'Ujjain', 'Mehsana',
       'Korba', 'Bahraich', 'Pali', 'Jehanabad', 'Erode', 'Ulhasnagar',
       'Pudukkottai', 'Coimbatore', 'Panchkula', 'Mau', 'Firozabad',
       'Ballia', 'Bathinda', 'Guntur', 'Bhilwara', 'South Dumdum',
       'Dhanbad', 'Farrukhab

***현재 city 값들부터 전체 점검***

In [14]:
df["area"].value_counts().reset_index().rename(
    columns={"area": "raw_city"}
)


,raw_city,count
0,Jalna,18
1,Deoghar,17
2,Bathinda,17
3,Orai,17
4,Ratlam,16
5,Ghaziabad,16
6,Kishanganj,15
7,Kadapa,14
8,Nandyal,14
9,Kalyan-Dombivli,14


***기본 정규화 작업(소문자, 공백 제거)***

In [15]:
df["city_norm"] = (
    df["area"]
    .str.lower()
    .str.strip()
)

df["area"]

0                              Udupi
1                            Aligarh
2                          Begusarai
3                          Kozhikode
4                       Ichalkaranji
5                      Visakhapatnam
6                            Gwalior
7                               Orai
8                              Buxar
9                           Malegaon
10                             Katni
11                          Fatehpur
12                      Ichalkaranji
13                         Hyderabad
14                          Bhiwandi
15                    Mira-Bhayandar
16                          Amravati
17                             Kochi
18                   Tiruchirappalli
19                     Hubli–Dharwad
20                         Nagercoil
21                             Alwar
22                         Bardhaman
23                           Nandyal
24                             Hapur
25                            Nanded
26                           Ramgarh
2

***특수 문자, 불필요한 문자 제거***

In [16]:

import re

df["city_norm"] = (
    df["city_norm"]
    .str.replace(r"[^\w\s]", "", regex=True)
)

***city 표준 매핑 딕셔너리 만들기(광역권 기준)***

In [24]:
city_map = {
    # Delhi NCR
    "delhi": "delhi",
    "new delhi": "delhi",
    "gurgaon": "delhi",
    "faridabad": "delhi",
    "noida": "delhi",
    "ghaziabad": "delhi",

    # Mumbai Metro
    "mumbai": "mumbai",
    "navi mumbai": "mumbai",
    "thane": "mumbai",
    "kalyan-dombivli": "mumbai",
    "vasai-virar": "mumbai",
    "bhiwandi": "mumbai",

    # Bangalore Metro
    "bangalore": "bangalore",
    "bengaluru": "bangalore",

    # Hyderabad Metro
    "hyderabad": "hyderabad",
    "secunderabad": "hyderabad",

    # Chennai Metro
    "chennai": "chennai",
    "avadi": "chennai",
    "ambattur": "chennai",
}


# 맵핑 안 된 도시들은 그대로 두기
df["city_std"] = df["city_norm"].replace(city_map)

***데이터 원본과 표준 비교***

In [29]:
city_std_df = df[["area", "city_norm", "city_std"]].drop_duplicates()

city_std_df.to_csv('city_map.csv', index=False, encoding='utf-8-sig')


---

***위도/경도 맵핑하기***

In [30]:
# 광역권 기준, 날씨 조회용 기준점
## Dataframe으로 한 이유
## 1. JOIN 구조 통일
## 2. CSV로 빼기 쉬움
## 3. 나중에 city 늘어나도 관리 편함

city_latlon = pd.DataFrame([
    {"city_std": "delhi", "lat": 28.6139, "lon": 77.2090},
    {"city_std": "mumbai", "lat": 19.0760, "lon": 72.8777},
    {"city_std": "bangalore", "lat": 12.9716, "lon": 77.5946},
    {"city_std": "hyderabad", "lat": 17.3850, "lon": 78.4867},
    {"city_std": "chennai", "lat": 13.0827, "lon": 80.2707},
])

# city_std_df에 붙이기
city_std_df = city_std_df.merge(city_latlon, on="city_std", how="left")

# 확인하기
## 여기서 NaN 뜨는 city가 있다면 → 광역권 매핑에서 빠진 도시라는 뜻 (정상적인 체크 포인트)
city_std_df[["city_std", "lat", "lon"]].drop_duplicates()


,city_std,lat,lon
0,udupi,NaN,NaN
1,aligarh,NaN,NaN
2,begusarai,NaN,NaN
3,kozhikode,NaN,NaN
4,ichalkaranji,NaN,NaN
5,visakhapatnam,NaN,NaN
6,gwalior,NaN,NaN
7,orai,NaN,NaN
8,buxar,NaN,NaN
9,malegaon,NaN,NaN


---

***Open-Meteo로 날씨 데이터 수집 코드***

In [ ]:
# 단일 도시, 기간 날씨 수집

import requests

def fetch_weather(lat, lon, start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_max",
            "temperature_2m_min",
            "precipitation_sum"
        ],
        "timezone": "Asia/Kolkata"
    }

    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()

    weather_df = pd.DataFrame({
        "date": data["daily"]["time"],
        "temp_max": data["daily"]["temperature_2m_max"],
        "temp_min": data["daily"]["temperature_2m_min"],
        "rain_sum": data["daily"]["precipitation_sum"],
    })

    return weather_df


***모든 city_std에 대해 날씨 수집***

In [ ]:
weather_all = []

for _, row in city_latlon.iterrows():
    city = row["city_std"]
    lat = row["lat"]
    lon = row["lon"]

    w = fetch_weather(
        lat=lat,
        lon=lon,
        start_date="2023-03-01",
        end_date="2024-11-30"
    )

    w["city_std"] = city
    weather_all.append(w)

weather_df = pd.concat(weather_all, ignore_index=True)
